# BASE DE DATOS
-------------------------------------------

## MONGO DB
------------

Mongo DB es una base de datos no relacional. Toda la información necesaria a su instalación se puede encontrar en: [Instalación MongoDB en Ubuntu](https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/)

Una vez iniciado mongoDB en local podemos acceder al mismo empleando la URI local: `mongodb://localhost:27017`.

Para manejar mongoDB en python debemos de instalar la libreria pymongo y importar MongoClient.

Un ejemplo de como iniciar mongo en python:

```
from pymongo import MongoClient

# connect with the client
Mongo_URI = 'mongodb://localhost:27017'
client = MongoClient(Mongo_URI)

# connect with the database
db = client.<databasename>

# connect with a collection
collection = db.<collectionname>
```

Dentro de cada colección encontramos documetos que simbolizaría cada entrada en la base de datos. Por ejemplo, en la base de datos *Northwind* encontramos la colección *Customers*, con documentos haciendo referencia a cada uno de esos clientes.

In [1]:
from pymongo import MongoClient, collation
from bson.objectid import ObjectId
from pprint import pprint

MONGO_URI="mongodb://localhost:27017"
client = MongoClient(MONGO_URI)
db = client.Northwind
collection = db.Customers

print("\nHas accedido con éxito a la colección clientes")
print(f"El objeto colección : \n{collection}")


Has accedido con éxito a la colección clientes
El objeto colección : 
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Northwind'), 'Customers')


### BUSQUEDA EN MONGO:

Una vez hemos cargado un objeto que nos permite conectarnos con la base de datos a la colección *Customers* podemos realizar búsquedas empleando los siguientes comandos:

1. \<CollectionObject\>.find({}) : toma un objeto tipo diccionario con los criterios de búsqueda y devuelve un cursor con todos los resultados que satisfagan los criterios especificados.

1. \<CollectionObject\>.find_one({}) : similar a find pero solo devuelve el primer resultado que encuentra en la base de datos.

1. \<CollectionObject\>.aggregate([]) : recibe una colección de objetos y añadir varios filtros para buscar en varias bases de datos o con varios criterios distintos.

**¿Como construir los criterios de busqueda?**:

Ejemplo de como buscar todos los clientes de un país:

```
{
    "Country": "USA"
}
```

Ejemplo de como excluir a los clientes de un país:

```
{
    "Country": { "$ne" : "USA"}
}
```

In [ ]:
from pymongo import MongoClient, collation
from bson.objectid import ObjectId
from pprint import pprint
import sys
import json

MONGO_URI="mongodb://localhost:27017"
client = MongoClient(MONGO_URI)
db = client.Northwind
collection = db.Customers

cursor = collection.find({"Country": "USA"})

print(f"Numero de documentos encontrados:  {cursor.count()}")
print("Numero de documentos encontrados: ", collection.count_documents({"Country": "USA"}))

print("Datos pendientes de leer:", cursor.alive)

while(cursor.alive):
    pprint(cursor.next()) # se mueve por el cursor

print("Datos pendientes de leer:", cursor.alive)

cursor = collection.find({"Country": "USA"}).limit(3) # los 3 primeros
cursor = collection.find({"Country": "USA"}).skip(5) # los 5 primeros saltalos
cursor = collection.find({"Country": "USA"}).skip(10).limit(10) # salta 10 y lee los 10 siguientes

cursor = collection.find({"Country": "USA"}).sort("City", 1) # en orden A to W
cursor = collection.find({"Country": "USA"}).sort("City", -1) # en orden W to A

**Listado de operadores relacionales**

1. $eq : equal
1. $lt : lower than
1. $lte : lower equal
1. $gt : greater than
1. $gte : greater equal
1. $ne : not equal
1. $in : in
1. $nin : not in

Todos estos operadores entre otros más se pueden usar para realizar una búsqueda con ciertas condiciones. 

In [ ]:
# EJERCICIO:
"""
Quiero buscar los clientes de Mexico
Mostrar su informacion : Nombre de la empresa
Mostrar los pedidos que tienen : CustomerID -> Pedidos
"""

Clientes = db.Customers
Pedidos = db.Orders

cursorClientes = Clientes.find({"Country": "Mexico"})

i = 0
while(cursorClientes.alive):
    i += 1
    d = cursorClientes.next()
    print(f"\nCLIENTE # {i}")
    print(d['ContactName'],"  :  " ,d['CompanyName'])
    cursorPedidos = Pedidos.find({"CustomerID": d['CustomerID']})
    ii = 0
    while(cursorPedidos.alive):
        ii += 1
        d2 = cursorPedidos.next()
        print("Pedido #"+str(ii)+" >> orderID :  "+d2['OrderID']+" |  Fecha : "+d2['OrderDate'])


# Obtener un cliente y todos sus pedidos en una única consulta

cursor = Clientes.aggregate([
    {"$match": {"CustomerID": "ANATR"}},
    {"$sort": {"City" : 1}},
    {"$lookup": {
        "from": "Orders",
        "localField" : "CustomerID",
        "foreignField": "CustomerID",
        "as": "Pedidos"
    }}
])

print()
while(cursor.alive):
    customer = cursor.next()
    print(customer['ContactName'],"  :  " , customer['CompanyName'])
    for ii in customer['Pedidos']:
        print(" >> orderID :  "+ii['OrderID']+" |  Fecha : "+ii['OrderDate'])

### AÑADIR NUEVOS REGISTROS:

Para añadir nuevos registros empleamos la función *insert_one()* o *insert_many([...])*. Para insertar creamos un diccionario o una clase y les damos los valores. Ej.

```
from pymongo import MongoClient, collation
from bson.objectid import ObjectId
from pprint import pprint
import sys
import json

class Customer:
    CustomerID = None
    CompanyName = None
    ContactName = None
    ContactTitle = None
    Address = None
    City = None
    Country = None
    PostalCode = None
    Region = None
    Phone = None
    Fax = None  


MONGO_URI="mongodb://localhost:27017"
client = MongoClient(MONGO_URI)
db = client.Northwind
collection = db.Customers

cliente = Customer()
cliente.CustomerID = "DEM10"
cliente.CompanyName = "Un dos tres bebidas SL"
cliente.ContactName = "Andres"
cliente.ContactTitle = "Propietario"
cliente.Address = "Gran Via, 42"
cliente.Region = "Madrid"
cliente.City = "Madrid"
cliente.Country = "Spain"
cliente.PostalCode = "28044"
cliente.Phone = "912002010"
cliente.Fax = "912002011"

print(cliente)

# transforma un objeto en un diccionario
print(cliente.__dict__)

resultado = collection.insert_one(cliente.__dict__)
```

### ACTUALIZAR REGISTROS

De forma similar a cuando insertamos, tenemos las funciones *update_one({query}, {newValues})* o *update_many([oldValues], [newValues])*. 

```
from pymongo import MongoClient, collation
from bson.objectid import ObjectId
from pprint import pprint
import sys
import json

MONGO_URI="mongodb://localhost:27017"
client = MongoClient(MONGO_URI)
db = client.Northwind
collection = db.Customers

query = {"Country": "EEUU"}

newValues = {
    "$set" : {
        "Country": "USA"
    }
}

resultado = collection.update_many(query, newValues)

print(resultado.matched_count, "documentos encontrados")
print(resultado.modified_count, "documentos modificados")
print("Estado de la operacion", resultado.acknowledged)
print()
print(collection.count_documents(query), "Clientes EEUU")
print(collection.count_documents({"Country": "USA"}), "Clientes USA")
```

## SQLServer
-----------

Es una base de datos relacional producida por Microsoft para Windows, Linux y Docker. El lenguaje de SQLServer se llama Transact-SQL. Es prácticamente igual que el standard.

[Transact-SQL](learn.microsoft.com/en-us/sql/t-sql/language-reference?view=sql-server-ver16)

### INSTALAR:

Para instalar el módulo necesario para trabajar necesitamos instalar el módulo *pymssql*. Para instalarlo introduce el siguiente comando:

```
pip install pymssql
```

### ESTABLECER CONEXION:

```
# Establecer conexion
connection = pymssql.connect( server = "",
                              user = "",
                              password = "",
                              database = ""
                            )

# Creamos un Cursor para ejecutar comando en BD
# Retorna tuplas
cursor = connection.cursor()
```


### EJECUTAR COMANDO SELECT

Realizamos una consulta a la base de datos con la función *execute(query)* donde la query es un string donde indicamos a la base de datos el comando a realizar. La base de datos emplea la sintaxis de Transact-SQL.

```
SELECT [columna1], [columna2], [columna3] FROM [nombreTabla] WHERE [condicion] ORDER BY [columna]
```

Aquí mostramos un ejemplo de una búsqueda:

```
# Ordenar por ciudad
cursor.execute("SELECT CustomerID, CompanyName, Country, City FROM dbo.Customers WHERE Country = %d ORDER BY City", "Spain")

# mostramos el contenido del cursor mediante FOR
for row in cursor.fetchall():
    print(f"     ID : {row['CustomerID']}")
    print(f"Empresa : {row['CompanyName']}")
    print(f"Empresa : {row['Country']}")
    print(f"Empresa : {row['City']}\n")
```

### EJECUTAR COMANDO INSERT

El comando para insertar valores a la base de datos es:

```
INSERT INTO [tableName]([column1], [column2], [column3]) VALUES('Value1', 'Value2', 'Value3')
```

Es importante tener en cuenta que las columnas tienen restricciones, como claves no nulas, únicas, etc. 